In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
from snorkel import SnorkelSession
session = SnorkelSession()

In [ ]:
from snorkel.models import candidate_subclass

Indicator = candidate_subclass('Indicator', ['trend'], ['indicator'])

In [ ]:
from snorkel.annotations import load_marginals

train_marginals = load_marginals(session, split=0)
train_cands = session.query(Indicator).filter(Indicator.split == 0).order_by(Indicator.id).all()
dev_cands   = session.query(Indicator).filter(Indicator.split == 1).order_by(Indicator.id).all()
test_cands  = session.query(Indicator).filter(Indicator.split == 2).order_by(Indicator.id).all()

In [ ]:
from snorkel.annotations import load_gold_labels

L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)

In [ ]:
from snorkel.learning.pytorch import LSTM

train_kwargs = {
    'lr':            0.01,
    'embedding_dim': 50,
    'hidden_dim':    50,
    'n_epochs':      10,
    'dropout':       0.25,
    'seed':          1701
}

lstm = LSTM(n_threads=None)
lstm.train(train_cands, train_marginals, X_dev=dev_cands, Y_dev=L_gold_dev, **train_kwargs)

In [ ]:
p, r, f1 = lstm.score(test_cands, L_gold_test)
print("Prec: {0:.3f}, Recall: {1:.3f}, F1 Score: {2:.3f}".format(p, r, f1))

In [ ]:
from snorkel.viewer import SentenceNgramViewer
tp, fp, tn, fn = lstm.error_analysis(session, test_cands, L_gold_test)
sv = SentenceNgramViewer(fp, session)
sv

In [ ]:
from snorkel.viewer import SentenceNgramViewer
#sv = SentenceNgramViewer(fn, session)
#sv
s = lstm.marginals(tmp_cands)
# s2 = lstm.predictions(tmp_cands)
# sv = SentenceNgramViewer(, session)
# sv
lstm.save_marginals(session, tmp_cands)
#lstm.save_marginals(session, unseen_cands)
# use the marginals here and extract 
# tmp_cands = session.query(Trend).filter(Trend.split == 3).order_by(Trend.id).all()
# print(len(tmp_cands))
# print(lstm.predictions(tmp_cands))
sv = SentenceNgramViewer(tn, session)
sv